In [ ]:
import torch 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as ohe
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("../../data/raw/obesity_prediction.csv")

In [ ]:
df.head(3)

In [ ]:
enc = ohe(handle_unknown="warn",sparse_output=False).set_output(transform="pandas")

In [ ]:
enc.fit(np.asarray(df["Gender"]).reshape(-1,1))

In [ ]:
enc.categories_

In [ ]:
gender_enc_df = enc.transform(np.asarray(df["Gender"]).reshape(-1,1))

In [ ]:
gender_enc_df

In [ ]:
gender_enc_df.columns =["female","male"]

In [ ]:
gender_enc_df.head()

In [ ]:
enc2 = ohe(sparse_output=False,handle_unknown="ignore").set_output(transform="pandas")

In [ ]:
df_mtran = enc2.fit_transform(np.asarray(df["MTRANS"]).reshape(-1,1))

In [ ]:
df_mtran.head()

In [ ]:
enc2.categories_

In [ ]:
df_mtran.columns = ["automobile","bike","motorbike","public_transportation","walking"]

In [ ]:
df_mtran.head(3)

In [ ]:
df_mtran.head()

In [ ]:
gender_enc_df.head()

In [ ]:
df_x = pd.concat(objs=[gender_enc_df,df_mtran,df["Age"],df["Height"]],axis=1)

In [ ]:
df_x

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
y_enc = le.fit_transform(df["Obesity"])

In [ ]:
y_enc

In [ ]:
le.classes_

In [ ]:
Y = torch.tensor(y_enc)

In [ ]:
Y.shape

In [ ]:
scaler = StandardScaler().set_output(transform="pandas")

In [ ]:
df_age_scaled = scaler.fit_transform(np.asarray(df["Age"]).reshape(-1,1))
df_height_scaled = scaler.fit_transform(np.asarray(df["Height"]).reshape(-1,1))

In [ ]:
df_height_scaled.columns =["height"]
df_age_scaled.columns = ["age"]

In [ ]:
df_height_scaled

In [ ]:
df_x = df_x.drop(labels=["Age","Height"],axis=1)
df_x.head()

In [ ]:
df_x = pd.concat(axis=1,objs=[df_x,df_age_scaled,df_height_scaled])

In [ ]:
df_x.head()

In [ ]:
class ffn(torch.nn.Module):
    
    def __init__(self,input_dims):
        super(ffn,self).__init__()    
        
        self.relu = torch.nn.ReLU()

        self.layer1 = torch.nn.Linear(input_dims,100)
        self.layer2 = torch.nn.Linear(100,50)
        self.layer3 = torch.nn.Linear(50,7)
    def forward(self,input):
        
        out = self.layer1(input)
        out = self.relu(self.layer2(out))
        out = self.relu(self.layer3(out))

        return out

In [ ]:
Y

In [ ]:
X = torch.tensor(np.asarray(df_x))

In [ ]:
X.shape

In [ ]:
df_x.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
class data(torch.utils.data.Dataset):

    def __init__(self, X,y):
        self.x = X
        self.y = y
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        record = self.x[index]
        label = self.y[index]
        return record,label

In [ ]:
len(X_train[0])

In [ ]:
model = ffn(9)

In [ ]:
train_dataset = data(X_train,y_train)
test_dataset = data(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
model.train()
epochs = 50

optim = torch.optim.AdamW(model.parameters(),lr = 0.001)

loss_fn = torch.nn.CrossEntropyLoss()
y_true = []
y_pred = []

for epoch in range(epochs):
    loss_per_epoch = 0
    for batch in train_loader:
        
        inputs,labels = batch
        optim.zero_grad()
        
        outputs = model(inputs.float())
        loss    = loss_fn(outputs,labels)
       
        _,preds = torch.max(outputs,1)
    
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())
        loss.backward()
        optim.step()
        
        loss_per_epoch = loss.item()
    
    print(f"Epoch: {epoch + 1}   Loss: {loss_per_epoch}")
    print(classification_report(y_true,y_pred))

In [ ]:
model.eval()

with torch.no_grad():
    test_loss = 0
    for batch in test_loader:
        
        inputs,labels = batch
        outputs = model(inputs.float())
        loss = loss_fn(outputs,labels)
        test_loss = loss.item()

    print(f"Test Loss: {test_loss}")
    
    

In [ ]:
a = torch.randn(10,3)
print(a)
val, idx = torch.max(a,1)

In [ ]:
idx

In [ ]:
b = torch.randn(5)

In [ ]:
torch.max(b,0)


In [ ]:
type(X_train)

In [ ]:
dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0}

for cls in y_train:

    y = cls.item()

    dict[y] += 1

In [ ]:
dict